<h1>Анализ сервиса для чтения книг<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Загрузка-и-обзор-данных" data-toc-modified-id="Загрузка-и-обзор-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка и обзор данных</a></span></li><li><span><a href="#Исследование" data-toc-modified-id="Исследование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование</a></span></li><li><span><a href="#Вывод-и-рекомендации" data-toc-modified-id="Вывод-и-рекомендации-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод и рекомендации</a></span><ul class="toc-item"><li><span><a href="#В-ходе-исследования-было-обнаружено" data-toc-modified-id="В-ходе-исследования-было-обнаружено-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>В ходе исследования было обнаружено</a></span></li><li><span><a href="#Рекомендации" data-toc-modified-id="Рекомендации-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Рекомендации</a></span></li></ul></li></ul></div>

## Описание проекта

**Область исследования**:\
База данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

**Задача**:\
Проанализировать базу с целью формирования ценностного предложения для продукта, в основу которого она ляжет - приложение для тех, кто любит читать.

**Структура данных:**
![Схема БД](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)

**Таблица books**\
Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

**Таблица authors**\
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

**Таблица publishers**\
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;


**Таблица ratings**\
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.


**Таблица reviews**\
Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

## Загрузка и обзор данных

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

Подключимся к базе данных:

In [2]:
#устанавливаем параметры
db_config = {
    'user': 'praktikum_student',  #имя пользователя
    'pwd': 'Sdf4$2;d-d30pp',  #пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,  #порт подключения
    'db': 'data-analyst-final-project-db'
}  #название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(
    **db_config)

#сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode': 'require'})

Сделаем пробный запрос из таблицы 'books', чтобы проверить соединение:

In [3]:
query = '''
SELECT * 
FROM books 
LIMIT 5;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Исследование

Поймем, сколько вообще книг  и отзывов на нах в нашей базе:

In [4]:
#книги
query = '''
SELECT COUNT (*)
FROM books;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,count
0,1000


In [5]:
#отзывы
query = '''
SELECT COUNT (*)
FROM reviews;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,count
0,2793


Определим диапазон дат публикации книг:

In [6]:
query = '''
SELECT 
MIN(EXTRACT(YEAR FROM publication_date)) AS earliest_year,
MAX(EXTRACT(YEAR FROM publication_date)) AS latest_year
FROM books;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,earliest_year,latest_year
0,1952.0,2020.0


Сколько книг вышло после 1 января 2000 года:

In [7]:
query = '''
SELECT 
COUNT(*) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,books_after_2000
0,819


Не смотря на разбег изданий с 1952 по 2020 год, почти 80% книг из базы вышли в последние 20 лет.

Определим для каждой книги количество обзоров и среднюю оценку:

In [8]:
query = '''
SELECT 
    b.book_id,
    b.title,
    COUNT(DISTINCT r.review_id) AS review_count,
    AVG(rt.rating) AS avg_rating
FROM 
    books AS b
LEFT JOIN 
    reviews AS r ON b.book_id = r.book_id
LEFT JOIN 
    ratings AS rt ON b.book_id = rt.book_id
GROUP BY 
    b.book_id, b.title
ORDER BY 
    review_count DESC;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,book_id,title,review_count,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Ожидаемо, в ТОПе по обзорам оказались бестселлеры последних лет.

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц (исключаем из анализа брошюры):

In [9]:
query = '''
SELECT 
    p.publisher_id,
    p.publisher,
    COUNT(b.book_id) AS book_count
FROM 
    books AS b
JOIN 
    publishers AS p ON b.publisher_id = p.publisher_id
WHERE 
    b.num_pages > 50
GROUP BY 
    p.publisher_id, p.publisher
ORDER BY 
    book_count DESC
LIMIT 1;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,publisher_id,publisher,book_count
0,212,Penguin Books,42


Т.к. информации о продажах у нас нет, можем только косвенно определять критерий популярности.\
Оценим, какой процент всех отзывов было оставлено на это издательство:

In [10]:
query = '''
WITH penguin_reviews AS (
    SELECT 
        r.review_id
    FROM 
        reviews AS r
    JOIN 
        books AS b ON r.book_id = b.book_id
    JOIN 
        publishers AS p ON b.publisher_id = p.publisher_id
    WHERE 
        p.publisher = 'Penguin Books'
),
total_reviews AS (
    SELECT 
        COUNT(*) AS total_review_count
    FROM 
        reviews
)
SELECT 
    (COUNT(pr.review_id)::FLOAT / (SELECT total_review_count FROM total_reviews)) * 100 AS percentage_reviews
FROM 
    penguin_reviews AS pr;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,percentage_reviews
0,4.61869


Сравним с другими издательствами, чтобы понять, насколько объем книг коррелирует с числом отзывов:

In [11]:
query = '''
WITH total_reviews AS (
    SELECT 
        COUNT(*) AS total_review_count
    FROM 
        reviews
)
SELECT 
    p.publisher AS publisher_name,
    (COUNT(r.review_id)::FLOAT / (SELECT total_review_count FROM total_reviews)) * 100 AS percentage_reviews,
    COUNT(DISTINCT b.book_id) AS unique_books_count
FROM 
    reviews AS r
JOIN 
    books AS b ON r.book_id = b.book_id
JOIN 
    publishers AS p ON b.publisher_id = p.publisher_id
GROUP BY 
    p.publisher
ORDER BY 
    percentage_reviews DESC
LIMIT 10;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,publisher_name,percentage_reviews,unique_books_count
0,Penguin Books,4.618690,42
1,Vintage,3.293949,31
2,Grand Central Publishing,2.398854,25
3,Penguin Classics,2.291443,24
4,Bantam,2.076620,19
5,St. Martin's Press,1.646975,14
6,Ballantine Books,1.611171,18
7,Harper Perennial,1.467956,12
8,Delta,1.396348,13
9,Back Bay Books,1.324740,11


Корреляция есть, но не такая прямая. Видим, что есть издательства с большим числом отзывов, но с меньшим количеством книг.

Определим авторов с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками):

In [12]:
query = '''
WITH author_book_ratings AS (
    SELECT
        authors.author AS author,
        books.book_id AS book_id,
        AVG(ratings.rating) AS avg_rating,
        COUNT(ratings.rating) AS count_rating
    FROM
        authors
    JOIN books ON books.author_id = authors.author_id
    JOIN ratings ON ratings.book_id = books.book_id
    GROUP BY
        authors.author,
        books.book_id
    HAVING
        COUNT(ratings.rating) >= 50)
SELECT
    author,
    AVG(avg_rating) AS avg_rating
FROM
    author_book_ratings
GROUP BY
    author
ORDER BY
    avg_rating DESC;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Узнаем, какие это книги данного автора и что за издательства их издавали:

In [13]:
query = '''
SELECT DISTINCT
    b.title,
    p.publisher
FROM
    books AS b
JOIN
    authors AS a ON b.author_id = a.author_id
JOIN
    publishers AS p ON b.publisher_id = p.publisher_id
WHERE
    a.author = 'Diana Gabaldon';
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,title,publisher
0,A Breath of Snow and Ashes (Outlander #6),Delta
1,Dragonfly in Amber (Outlander #2),Bantam
2,Drums of Autumn (Outlander #4),Delta
3,Lord John and the Private Matter (Lord John Gr...,Seal
4,Outlander (Outlander #1),Dell Publishing Company
5,Voyager (Outlander #3),Delta


Определим среднее количество обзоров от пользователей, которые поставили больше 48 оценок:

In [14]:
query = '''
WITH active_users AS (
    SELECT 
        username
    FROM 
        ratings
    GROUP BY 
        username
    HAVING 
        COUNT(rating_id) > 48
)
SELECT 
    AVG(user_reviews.review_count) AS avg_reviews
FROM (
    SELECT 
        r.username,
        COUNT(r.review_id) AS review_count
    FROM 
        reviews AS r
    WHERE 
        r.username IN (SELECT username FROM active_users)
    GROUP BY 
        r.username
) AS user_reviews;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,avg_reviews
0,24.0


Посмотрим, на книги каких авторов эти ревьюеры оставляют свои отзывы:

In [15]:
query = '''
WITH active_users AS (
    SELECT 
        username
    FROM 
        ratings
    GROUP BY 
        username
    HAVING 
        COUNT(rating_id) > 48
)
SELECT 
    a.author AS author_name,
    b.title AS book_title,
    r.username AS reviewer,
    COUNT(r.review_id) OVER (PARTITION BY r.username) AS review_count
FROM 
    reviews AS r
JOIN 
    books AS b ON r.book_id = b.book_id
JOIN 
    authors AS a ON b.author_id = a.author_id
ORDER BY 
    author_name
LIMIT 20;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,author_name,book_title,reviewer,review_count
0,A.S. Byatt,Possession,carriehale,16
1,A.S. Byatt,Possession,paul88,22
2,A.S. Byatt,Possession,alison92,14
3,Aesop/Laura Harris/Laura Gibbs,Aesop's Fables,ewerner,24
4,Aesop/Laura Harris/Laura Gibbs,Aesop's Fables,martinadam,27
5,Agatha Christie,The Body in the Library (Miss Marple #3),jessica49,16
6,Agatha Christie,The Mysterious Affair at Styles (Hercule Poiro...,jeromebowen,17
7,Agatha Christie,The Mysterious Affair at Styles (Hercule Poiro...,johnsonamanda,24
8,Agatha Christie,Murder at the Vicarage (Miss Marple #1),brandtandrea,19
9,Agatha Christie,The Murder of Roger Ackroyd (Hercule Poirot #4),laura42,17


Активные ревьюеры предпочитают детективы и интеллектуальную литературу с уклоном в филологию.

Определим, есть ли зависимость между количеством страниц книги и средней оценкой или количеством обзоров:

In [16]:
query = '''
SELECT
    CASE 
        WHEN num_pages <= 50 THEN '0-50'
        WHEN num_pages <= 100 THEN '51-100'
        WHEN num_pages <= 200 THEN '101-200'
        WHEN num_pages <= 300 THEN '201-300'
        WHEN num_pages <= 400 THEN '301-400'
        ELSE '400+'
    END AS page_range,
    COUNT(DISTINCT b.book_id) AS book_count,
    AVG(rt.rating) AS avg_rating
FROM
    books AS b
JOIN
    ratings AS rt ON b.book_id = rt.book_id
GROUP BY
    page_range
ORDER BY
    avg_rating DESC;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,page_range,book_count,avg_rating
0,0-50,8,4.033333
1,51-100,22,4.000000
2,201-300,212,3.948498
3,400+,358,3.938410
4,301-400,282,3.931530
5,101-200,118,3.839538


Удивительно, но высокие рейтинги у того сегмента, который мы изначально исключаем из нашего анализа - так называемые "брошюры".\
А что если это не они, а, например, детские книги.

In [17]:
query = '''
SELECT
    b.book_id,
    b.title,
    b.num_pages,
    a.author AS author_name,
    AVG(rt.rating) AS avg_rating
FROM
    books AS b
JOIN
    ratings AS rt ON b.book_id = rt.book_id
JOIN
    authors AS a ON b.author_id = a.author_id
WHERE
    b.num_pages <= 50
GROUP BY
    b.book_id, b.title, b.num_pages, a.author
ORDER BY
    avg_rating DESC;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,book_id,title,num_pages,author_name,avg_rating
0,801,The Monster at the End of this Book,32,Jon Stone/Michael J. Smollin,4.750000
1,988,Without Fail (Jack Reacher #6),14,Lee Child/Dick Hill,4.333333
2,286,Goodnight Moon,32,Margaret Wise Brown/Clement Hurd,4.300000
3,771,The Kissing Hand,32,Audrey Penn/Ruth E. Harper/Nancy M. Leak,4.000000
4,191,Disney's Beauty and the Beast (A Little Golden...,24,Teddy Slater/Ron Dias/Ric González,4.000000
5,855,The Road Not Taken and Other Poems,49,Robert Frost,3.500000
6,153,Cloudy With a Chance of Meatballs (Cloudy with...,32,Judi Barrett/Ron Barrett,3.400000
7,446,Moo Baa La La La!,14,Sandra Boynton,3.000000


Да, кажется стоит обратить более пристальное внимание на эсегмент детских книг, как минимум учитывать в анализе продаж.

Определим, есть ли зависимость между временем, прошедшим с публикации книги, и её средней оценкой:

In [18]:
query = '''
SELECT
    publication_range,
    COUNT(DISTINCT book_publication.book_id) AS book_count,
    AVG(r.rating) AS avg_rating
FROM (
    SELECT
        book_id,
        title,
        CASE 
            WHEN publication_date <= '2000-12-31' THEN 'Before 2000'
            WHEN publication_date BETWEEN '2001-01-01' AND '2015-12-31' THEN '2001-2014'
            WHEN publication_date BETWEEN '2015-01-01' AND '2020-12-31' THEN '2015-2020'
            ELSE 'Other'
        END AS publication_range
    FROM
        books
) AS book_publication
JOIN
    ratings AS r ON book_publication.book_id = r.book_id
GROUP BY
    publication_range
ORDER BY
    avg_rating DESC;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,publication_range,book_count,avg_rating
0,Before 2000,217,3.956556
1,2001-2014,780,3.921489
2,2015-2020,3,3.833333


Интересно отметить, что, в нашей выборке, более старые книги имеют более высокий рейтинг. Возможно это пропорционально количесву времени, прошедшему с релиза.\
Но что примечательнее, три книги, которые вышли в последние 5 лет, имеют не слишком отличающийся рейтинг от тех книг, которым уже 68 лет.

Нужно понять, на каких бестсеелерах стоит концентрироваться.\
Выведем ТОП-10 книг самым большим количеством оценок выше 4.5 (учитываем только книги с 10+ оценкок):

In [19]:
query = '''
SELECT
    b.book_id,
    b.title,
    COUNT(r.rating) AS high_rating_count
FROM
    books AS b
JOIN
    ratings AS r ON b.book_id = r.book_id
WHERE
    r.rating > 4.5
GROUP BY
    b.book_id, b.title
HAVING
    COUNT(r.rating) >= 10
ORDER BY
    high_rating_count DESC
    LIMIT 10;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,book_id,title,high_rating_count
0,302,Harry Potter and the Prisoner of Azkaban (Harr...,48
1,299,Harry Potter and the Chamber of Secrets (Harry...,41
2,300,Harry Potter and the Half-Blood Prince (Harry ...,38
3,722,The Fellowship of the Ring (The Lord of the Ri...,37
4,301,Harry Potter and the Order of the Phoenix (Har...,32
5,948,Twilight (Twilight #1),32
6,750,The Hobbit or There and Back Again,32
7,656,The Book Thief,26
8,779,The Lightning Thief (Percy Jackson and the Oly...,24
9,399,Little Women,24


Всё так же бестселлеры последних лет и классик фэнтези.

Эти книги стоит промоутировать не так активно: ТОП книг с самыми большим количеством оценок ниже 3 (учитываем только книги с 8+ оценкок):

In [20]:
query = '''
SELECT
    b.book_id,
    b.title,
    COUNT(r.rating) AS low_rating_count
FROM
    books AS b
JOIN
    ratings AS r ON b.book_id = r.book_id
WHERE
    r.rating < 3
GROUP BY
    b.book_id, b.title
HAVING
    COUNT(r.rating) >= 8
ORDER BY
    low_rating_count DESC
LIMIT 10;
'''
con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,book_id,title,low_rating_count
0,948,Twilight (Twilight #1),14
1,75,Angels & Demons (Robert Langdon #1),13
2,488,Of Mice and Men,12
3,207,Eat Pray Love,12
4,673,The Catcher in the Rye,8
5,696,The Da Vinci Code (Robert Langdon #2),8


Похоже, Сумерки вызывают весь спектр эмоций, от любви до ненависти. Скорее всего, их нужно промоутировать, но на определенный сегмент.

## Вывод и рекомендации

### В ходе исследования было обнаружено

- Большинство книг из базы данных были изданы в последние 20 лет, что отражает активность издателей за этот период;
- Популярные бестселлеры последних лет (например, Twilight, Harry Potter) получают наибольшее количество отзывов.
- Penguin Books — лидирует по числу книг и отзывов, но зависимость между объемом книг и количеством отзывов не всегда прямая;
- J.K. Rowling имеет высокую среднюю оценку своих книг, что подтверждает её популярность среди читателей. Это же подтверждает ТОП-10 книг самым большим количеством оценок - всюду Гарри Поттер;
- Активные ревьюеры предпочитают детективы и интеллектуальную литературу, особенно произведения Агаты Кристи;
- Книги с меньшим количеством страниц (до 50) показывают высокие рейтинги, что может свидетельствовать о популярности детских книг;
- Более старые книги (до 2000 года) имеют более высокие рейтинги, несмотря на наличие новых популярных релизов;
- Книги с низкими оценками, например, Twilight, вызывают широкий спектр эмоций и могут быть эффективны для целевых сегментов.

### Рекомендации

**Рекомендательная система:**
- Сегментировать пользователей по активности и жанрам, чтобы предложить персонализированные рекомендации (например, детективы, интеллектуальная литература и фэнтези для активных ревьюеров);
- Мотивировать активных пользователей оставлять больше отзывов через бонусы и персонализированные рекомендации;
- Таргетировать книги с высоким рейтингом (например, Penguin Books с рейтингом 4.6 и книги Diana Gabaldon с рейтингом 4.3).

**Стратегия наполнения базы:**
- Активно добавлять книги от Penguin Books, так как они получают высокие оценки и отзывы;
- Сосредоточиться на авторах с высокими рейтингами, такими как J.K. Rowling, чьи книги получают рейтинг 4.4;
- Разнообразить базу, включая больше детских книг, которые показывают высокие оценки (например, "The Monster at the End of this Book" с рейтингом 4.75);
- Включать как новые книги, так и классические произведения до 2000 года, так как они имеют высокие оценки (средний рейтинг 3.96 для книг до 2000 года).